<a href="https://colab.research.google.com/github/Brackly/Zindi-Swahili-News-Classification-Model/blob/main/SwahiliNews_Classification_Zindi_Finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installations

In [ ]:
!pip install transformers tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.4 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 2.0 MB/s eta 0:00:00


In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Downloading pretrained Model

In [ ]:
import torch
from transformers import BertTokenizer,BertModel, Trainer, TrainingArguments
name="Davlan/bert-base-multilingual-cased-finetuned-swahili"
tokenizer=BertTokenizer.from_pretrained(name,truncation_side='right')
model=BertModel.from_pretrained(name)

Some weights of BertModel were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-swahili and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Loading dataset

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
#Please change this path to point to your csv file
train_path='/content/drive/MyDrive/chatgpt-ml-challenge-swahili-news-classification20230706-4525-yqsxo1/Train.csv'

In [ ]:
import pandas as pd
df=pd.read_csv(train_path)
df.head()

,id,content,category
0,SW0,SERIKALI imesema haitakuwa tayari kuona amani...,Kitaifa
1,SW1,"Mkuu wa Mkoa wa Tabora, Aggrey Mwanri amesiti...",Biashara
2,SW10,SERIKALI imetoa miezi sita kwa taasisi zote z...,Kitaifa
3,SW100,KAMPUNI ya mchezo wa kubahatisha ya M-bet ime...,michezo
4,SW1000,WATANZANIA wamekumbushwa kusherehekea sikukuu...,Kitaifa


In [ ]:
import torch
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self,text,label,tokenizer,block_size,device):
    self.label=label
    batch_encoding = tokenizer(text, add_special_tokens=True, padding=True,truncation=True, max_length=block_size)
    self.examples=[]
    for i in range(len(batch_encoding["input_ids"])):
      self.examples.append({"input_ids":torch.tensor(batch_encoding["input_ids"][i]).to(device),
                            'token_type_ids':torch.tensor(batch_encoding["token_type_ids"][i]).to(device),
                            'attention_mask':torch.tensor(batch_encoding["attention_mask"][i]).to(device)
                            })

  def __len__(self):
    return len(self.label)

  def __getitem__(self,idx):

    item = {'inputs':self.examples[idx]}
    item['label'] = torch.FloatTensor(self.label[idx]).to(device)
    return item


In [ ]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
idx = int(len(df)*0.8)
train_dataset = CustomDataset(list(df['content'].values)[:idx],
                        list(pd.get_dummies(df['category']).values[:idx]),
                        tokenizer,
                        512,device)
eval_dataset = CustomDataset(list(df['content'].values)[idx:],
                        list(pd.get_dummies(df['category']).values[idx:]),
                        tokenizer,
                        512,device)

In [ ]:
print(train_dataset.__getitem__(0))

{'inputs': {'input_ids': tensor([   101,  23056,  46876,  67813,  11369,  11281,  19960,  38443,  10113,
         13080,  19049, 104388,  30562,  12476,  15694,  12920,  28149,  10342,
         10132,  11735,  24207,  13048,  11471,  47622,  22944,  11119,  82351,
         10113,  26506,  10853,  12322,  13109,  13903,  10637,    189,  12577,
         13520,  11471,  10293,  16326,  46488,  10549,  11471,  13470,  12806,
         12517,  15694,  49101,  10114,  78464,  10116,  10549,  13872,  10342,
           117,  77423,  10330,  10257,  10271,  13060,  10703,  13672,  70436,
           119,  27512,  10133,  10549,  36376,  10147,  11037,  10132,  10685,
         91610,    160,  28059,  10401,  11471,  13744,  25287,  10549,    151,
         81722,  10549,    151,  12806,    117,  39135,  11488,  19387, 102670,
         10575,  64807,  23124,  61450,  17657,  47513,  11908,  15900,  10219,
         10549, 107928,  38501,  10390,  90793,  18028,  69513,  35368,  10196,
         27162,

In [ ]:
from torch.utils.data import DataLoader
batch_size = 32
shuffle = True
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
eval_data_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=shuffle)

### Finetuning the model

In [ ]:
class SwahiliZindiFinetuned(torch.nn.Module):
  def __init__(self,model,num_classes):
    super().__init__()
    div=8
    hidden_size=model.config.hidden_size
    self.ln1 = torch.nn.Linear(hidden_size,int(hidden_size/div))
    self.ln2 = torch.nn.Linear(int(hidden_size/div),int(hidden_size/(div*div)))
    self.classifier = torch.nn.Linear(int(hidden_size/(div*div)),num_classes)
    self.actfn=torch.nn.functional
    self.bert=model
    self.actv_fn = torch.nn.Tanh()
    self.lossfn=torch.nn.CrossEntropyLoss()
    for param in self.bert.parameters():
      param.requires_grad= False

  def forward(self,input_ids,attention_mask):
    output = self.bert(input_ids=input_ids,attention_mask=attention_mask).pooler_output
    output = self.actv_fn(self.ln1(output))
    output = self.actv_fn(self.ln2(output))
    output= self.actv_fn(self.classifier(output))
    return output
zindi_model = SwahiliZindiFinetuned(model,len(set(df['category'])))
zindi_model.to(device)

SwahiliZindiFinetuned(
  (ln1): Linear(in_features=768, out_features=96, bias=True)
  (ln2): Linear(in_features=96, out_features=12, bias=True)
  (classifier): Linear(in_features=12, out_features=5, bias=True)
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
   

In [ ]:
# PATH='/content/drive/MyDrive/Zindi_Model/model_v2.pt'


# checkpoint = torch.load(PATH)
# zindi_model.load_state_dict(checkpoint['model_state_dict'])
# # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']

# optimizer = torch.optim.Adam(zindi_model.parameters(),lr=1e-4)
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

### Custom training loop

In [ ]:
# Set the model to training mode
zindi_model.train()
num_epochs=15


# Loop over the epochs
for epoch in range(num_epochs):

  # Initialize the running loss and accuracy
  running_loss = 0.0
  running_acc = 0.0

  # Loop over the batches
  for batch in train_data_loader:

    # Move the inputs and labels to the device
    inputs = batch['inputs']
    labels = batch['label']

    # Zero the parameter gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = zindi_model(input_ids=inputs['input_ids'],attention_mask=inputs['attention_mask'])

    # Compute the loss
    loss = zindi_model.lossfn(outputs, labels)

    # Backward pass and optimize
    loss.backward()
    optimizer.step()

    # Update the running loss and accuracy
    running_loss += loss.item()
    running_acc += (outputs.argmax(dim=-1) == labels.argmax(dim=-1)).float().mean().item()

  # Print the epoch statistics
  total_loss =running_acc
  total_accuracy = running_loss
  print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_data_loader):.4f},Accuracy: {running_acc/len(train_data_loader):.4f}'
        )
print(f'Loss: {running_loss/len(train_data_loader):.4f}')
print(f'Accuracy: {running_acc/len(train_data_loader):.4f}')

NameError: ignored

### Save the model

In [ ]:
PATH='/content/drive/MyDrive/Zindi_Model/model_v2.pt'
torch.save({
            'epoch': epoch,
            'model_state_dict': zindi_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, PATH)

### Model Evaluation

In [ ]:
zindi_model.eval()
num_epochs=10
# Loop over the epochs
optimizer = torch.optim.Adam(zindi_model.parameters(),lr=1e-4)
total_loss=0
total_accuracy=0
with torch.no_grad():
  for epoch in range(num_epochs):
    # Initialize the running loss and accuracy
    running_loss = 0.0
    running_acc = 0.0
    # Loop over the batches
    for batch in eval_data_loader:
      # Move the inputs and labels to the device
      inputs = batch['inputs']
      labels = batch['label']

      # Zero the parameter gradients
      optimizer.zero_grad()

      # Forward pass
      outputs = zindi_model(input_ids=inputs['input_ids'],attention_mask=inputs['attention_mask'])
      running_loss += loss.item()
      running_acc += (outputs.argmax(dim=-1) == labels.argmax(dim=-1)).float().mean().item()

    # Print the epoch statistics
    total_loss += (running_loss/len(eval_data_loader))
    total_accuracy += (running_acc/len(eval_data_loader))
print(f'Total loss: {total_loss/num_epochs}')
print(f'Total Accuracy: {total_accuracy/num_epochs}')

Epoch 1, Loss: 0.5626,Accuracy: 0.8343
Epoch 2, Loss: 0.5626,Accuracy: 0.8241
Epoch 3, Loss: 0.5626,Accuracy: 0.8309
Epoch 4, Loss: 0.5626,Accuracy: 0.8275
Epoch 5, Loss: 0.5626,Accuracy: 0.8275
Epoch 6, Loss: 0.5626,Accuracy: 0.8309
Epoch 7, Loss: 0.5626,Accuracy: 0.8309
Epoch 8, Loss: 0.5626,Accuracy: 0.8309
Epoch 9, Loss: 0.5626,Accuracy: 0.8241
Epoch 10, Loss: 0.5626,Accuracy: 0.8309
Total loss: 0.5625878572463989
Total Accuracy: 0.8292072523723949


### Predictions

In [ ]:
pred_df=pd.read_csv('/content/drive/MyDrive/chatgpt-ml-challenge-swahili-news-classification20230706-4525-yqsxo1/Test.csv')
pred_df.head()

,swahili_id,content
0,001dd47ac202d9db6624a5ff734a5e7dddafeaf2,"MKUU wa Wilaya ya Bahi, Mkoani Dodoma, Mwanah..."
1,0043d97f7690e9bc02f0ed8bb2b260d1d44bad92,"MWISHONI mwa wiki hii, Timu ya Soka ya Taifa,..."
2,00579c2307b5c11003d21c40c3ecff5e922c3fd8,THAMANI ya mauzo ya bidhaa za Afrika Masharik...
3,00868eeee349e286303706ef0ffd851f39708d37,MENEJA Mawasiliano na Utetezi wa asasi ya AGP...
4,00a5cb12d3058dcf2e42f277eee599992db32412,"WAZIRI wa Kilimo, Japhet Hasunga amesema seri..."


In [ ]:
len(pred_df)

1288

In [ ]:
mapping={
    0:'Biashara',
    1:'Burudani',
    2:'Kimataifa',
    3:'Kitaifa',
    4:'michezo'
}

In [ ]:
inputs=tokenizer(list(pred_df['content'].values),padding=True,truncation=True, max_length=512,return_tensors='pt')

In [ ]:
import torch.nn.functional as F
preds=[]

for i in range(len(pred_df['content'])):
  output = F.softmax(zindi_model(input_ids=inputs['input_ids'][i].view(1,-1).to(device),
                       attention_mask=inputs['attention_mask'][i].view(1,-1).to(device)),
                     dim=-1)

  preds.append(mapping[torch.argmax(output).item()])
print(preds)

['Kitaifa', 'michezo', 'Biashara', 'Kitaifa', 'Kitaifa', 'michezo', 'Kitaifa', 'michezo', 'michezo', 'michezo', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'michezo', 'Kitaifa', 'Kitaifa', 'Biashara', 'Kitaifa', 'Biashara', 'michezo', 'michezo', 'Kitaifa', 'Kitaifa', 'michezo', 'Kitaifa', 'Kitaifa', 'Biashara', 'michezo', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Biashara', 'Biashara', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'michezo', 'Kitaifa', 'michezo', 'michezo', 'Kitaifa', 'Biashara', 'Kitaifa', 'Kitaifa', 'michezo', 'Kitaifa', 'michezo', 'michezo', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Biashara', 'Biashara', 'michezo', 'Kitaifa', 'Biashara', 'Biashara', 'Kitaifa', 'michezo', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Biashara', 'Kitaifa', 'michezo', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Biashara', 'Biashara', 'Kitaifa', 'michezo', 'Biashara', 'michezo', 'Kitaifa', 'Kitaifa', 'Biashara', 'Biashara', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'michezo', 'Kitaifa', 'michezo', 'Bi

In [ ]:
pred_df['predictions']=preds
pred_df.to_csv('./predictions.csv')

In [ ]:
new_preds=pd.get_dummies(pred_df['predictions'])

In [ ]:
new_preds['swahili_id']=pred_df['swahili_id']

In [ ]:
new_preds=new_preds.set_index('swahili_id')

In [ ]:
new_preds.to_csv('/content/predictions.csv')

In [ ]:
len(new_preds)

1288